In [1]:
from __future__ import unicode_literals

import json
import os

# %pip install -U hazm
from hazm import *

os.chdir('../')

def read_file():
    documents_title = []
    documents_content = []
    documents_url = []
    with open(os.path.join(os.getcwd(), 'Phase_1', 'assets', 'IR_data_news_12k.json'), encoding='UTF-8') as f:
        data = json.load(f)
        for i in data:
            documents_title.append(data[i]["title"])
            documents_content.append(data[i]["content"])
            documents_url.append(data[i]['url'])
    return documents_url, documents_title, documents_content


docs_url, docs_title, docs_content = read_file()
normalizer = Normalizer()

In [2]:
# coding: utf8
from os import path
import codecs
from hazm.Normalizer import Normalizer
# from Phase_1.src.utils import data_path
default_stop_words = path.join(os.getcwd(), 'Phase_1', 'src', 'data', 'stopwords.dat')


class StopWord:
    """ Class for remove stop words

         >>> StopWord().clean(["در","تهران","کی","بودی؟"])
         ['بودی؟', 'تهران', 'کی']
         >>> StopWord(normal=True).clean(["در","تهران","کی","بودی؟"])
         ['بودی؟', 'تهران']

         """

    def __init__(self, file_path=default_stop_words, normal=False):
        self.file_path = file_path
        self.normal = normal
        self.normalizer = Normalizer().normalize
        self.stop_words = self.init(file_path, normal)

    def init(self, file_path, normal):
        if not normal:
            return set(
                line.strip("\r\n") for line in codecs.open(file_path, "r", encoding="utf-8").readlines())
        else:
            return set(
                self.normalizer(line.strip("\r\n")) for line in
                codecs.open(file_path, "r", encoding="utf-8").readlines())

    def set_normalizer(self, func):
        self.normalizer = func
        self.stop_words = self.init(self.file_path, self.normal)

    def __getitem__(self, item):
        return item in self.stop_words

    def __str__(self):
        return str(self.stop_words)

    def clean(self, iterable_of_strings, return_generator=False):
        if return_generator:
            return filter(lambda item: not self[item], iterable_of_strings)
        else:
            return list(filter(lambda item: not self[item], iterable_of_strings))


In [3]:
def preprocess_content(content):
    str_empty = ' '
    stemmer = Stemmer()
    content = normalizer.normalize(content)
    content = word_tokenize(content)
    content = StopWord(normal=False).clean(content)
    content = [stemmer.stem(word) for word in content]
    content = str_empty.join(content)
    return content

In [86]:
for i in range(len(docs_content)):
    docs_content[i] = preprocess_content(docs_content[i])

In [87]:
from datetime import datetime
from elasticsearch import Elasticsearch, helpers

# Connect to 'http://localhost:9200'
es = Elasticsearch("http://localhost:55001")

In [90]:
import uuid

def generate_data():
    data = []
    for doc_content, doc_url, doc_title in zip(docs_content, docs_url, docs_title):
        if doc_content and doc_url and doc_title:        
            data.append({
                "_index": "ir-news-index",
                "_id" : uuid.uuid4(),
                "_source": {
                    "content": doc_content,
                    "url": doc_url,
                    "title": doc_title
                }
            })
    return data

data=generate_data()
print(docs_content[0])
response = helpers.bulk(es, data)

گزار خبرگزار فارس کنفدراسیون فوتبال آسیا AFC نامه رسم فدراسیون فوتبال ایر باشگاه گیت پسند قرعه کش جا باشگاه فوتسال آسیا اساس ۲۵ فروردین ماه ۱۴۰۱ مراس قرعه کش جا باشگاه فوتسال آسیا مالز برگزار باشگاه گیت پسند بعنو قهر فوتسال ایر سال ۱۴۰۰ مسابق کرده_اس گیت پسند تجربه ۳ دوره حضور جا باشگاه فوتسال آسیا سه دوره فینال مسابق قهرمان مقا دوم بدس آورده_اس انت پیام/
